In [ ]:
!pip install pytorch-lightning torchvision pillow tensorboard


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os

path = "/content/drive/MyDrive/anime_dataset/images"
print("Exists:", os.path.exists(path))
print("Total images:", len(os.listdir(path)))
print("First 5:", os.listdir(path)[:5])


In [ ]:
import os, glob
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


In [ ]:
class AnimeDataset(Dataset):
    def __init__(self, root_dir, max_images=10000, transform=None):
        self.image_paths = sorted(
            glob.glob(os.path.join(root_dir, "*.jpg"))
        )[:max_images]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img


In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

dataset = AnimeDataset(
    "/content/drive/MyDrive/anime_dataset/images",
    transform=transform
)

print("Images loaded:", len(dataset))


In [ ]:
data_loader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2
)

print("DataLoader ready")


In [ ]:
class Generator(nn.Module):
    def __init__(self, nz=100):
        super().__init__()
        self.nz = nz
        self.net = nn.Sequential(
            nn.ConvTranspose2d(nz, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x.view(x.size(0), self.nz, 1, 1))


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 1, 4, 1, 0),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).view(-1)


In [ ]:
import pytorch_lightning as pl


In [ ]:
class DCGAN(pl.LightningModule):
    def __init__(self, nz=100):
        super().__init__()
        self.nz = nz
        self.generator = Generator(nz)
        self.discriminator = Discriminator()
        self.loss = nn.BCELoss()
        self.automatic_optimization = False

    def training_step(self, batch, batch_idx):
        real = batch
        bs = real.size(0)
        opt_g, opt_d = self.optimizers()

        valid = torch.ones(bs, device=self.device)
        fake = torch.zeros(bs, device=self.device)

        z = torch.randn(bs, self.nz, device=self.device)
        fake_imgs = self.generator(z).detach()

        d_loss = (self.loss(self.discriminator(real), valid) +
                  self.loss(self.discriminator(fake_imgs), fake)) / 2

        opt_d.zero_grad()
        self.manual_backward(d_loss)
        opt_d.step()

        z = torch.randn(bs, self.nz, device=self.device)
        g_loss = self.loss(self.discriminator(self.generator(z)), valid)

        opt_g.zero_grad()
        self.manual_backward(g_loss)
        opt_g.step()

    def configure_optimizers(self):
        return [
            torch.optim.Adam(self.generator.parameters(), lr=2e-4),
            torch.optim.Adam(self.discriminator.parameters(), lr=2e-4)
        ]


In [ ]:
model = DCGAN()

trainer = pl.Trainer(
    max_epochs=5,   # keep small
    accelerator="cpu",  # GPU optional
    devices=1
)

trainer.fit(model, data_loader)


In [ ]:
import torchvision.utils as vutils
import matplotlib.pyplot as plt

z = torch.randn(32, 100)
with torch.no_grad():
    imgs = model.generator(z)

imgs = (imgs + 1) / 2
grid = vutils.make_grid(imgs, nrow=8)

plt.figure(figsize=(8,8))
plt.imshow(grid.permute(1,2,0))
plt.axis("off")
plt.show()


In [ ]:
vutils.save_image(imgs, "generated_anime_sample.png", nrow=8)
print("Saved generated_anime_sample.png")


In [ ]:
trainer = pl.Trainer(
    max_epochs=30,        # increase gradually
    accelerator="gpu",
    devices=1
)


In [ ]:
!pip install pytorch-lightning torchvision pillow


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os, glob
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.utils as vutils

import pytorch_lightning as pl
import matplotlib.pyplot as plt


In [ ]:
class AnimeDataset(Dataset):
    def __init__(self, root_dir, max_images=18000, transform=None):
        self.image_paths = sorted(
            glob.glob(os.path.join(root_dir, "*.jpg"))
        )[:max_images]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img


In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

dataset = AnimeDataset(
    "/content/drive/MyDrive/anime_dataset/images",
    max_images=18000,
    transform=transform
)

print("Total images used:", len(dataset))

data_loader = DataLoader(
    dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)


In [ ]:
class Generator(nn.Module):
    def __init__(self, nz=100):
        super().__init__()
        self.nz = nz
        self.net = nn.Sequential(
            nn.ConvTranspose2d(nz, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.net(x.view(x.size(0), self.nz, 1, 1))


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).view(-1)


In [ ]:
class DCGAN(pl.LightningModule):
    def __init__(self, nz=100):
        super().__init__()
        self.nz = nz
        self.generator = Generator(nz)
        self.discriminator = Discriminator()
        self.loss = nn.BCELoss()
        self.automatic_optimization = False

    def training_step(self, batch, batch_idx):
        real = batch
        bs = real.size(0)

        opt_g, opt_d = self.optimizers()

        valid = torch.ones(bs, device=self.device)
        fake = torch.zeros(bs, device=self.device)

        # ---- Train Discriminator ----
        z = torch.randn(bs, self.nz, device=self.device)
        fake_imgs = self.generator(z).detach()

        d_loss = (
            self.loss(self.discriminator(real), valid) +
            self.loss(self.discriminator(fake_imgs), fake)
        ) / 2

        opt_d.zero_grad()
        self.manual_backward(d_loss)
        opt_d.step()

        # ---- Train Generator ----
        z = torch.randn(bs, self.nz, device=self.device)
        gen_imgs = self.generator(z)
        g_loss = self.loss(self.discriminator(gen_imgs), valid)

        opt_g.zero_grad()
        self.manual_backward(g_loss)
        opt_g.step()

    def configure_optimizers(self):
        opt_g = torch.optim.Adam(self.generator.parameters(), lr=2e-4, betas=(0.5, 0.999))
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=2e-4, betas=(0.5, 0.999))
        return [opt_g, opt_d]


In [ ]:
model = DCGAN()

trainer = pl.Trainer(
    max_epochs=30,          # GOOD quality
    accelerator="gpu",
    devices=1
)

trainer.fit(model, data_loader)


In [ ]:
model.generator.eval()

z = torch.randn(32, 100, device=model.device)

with torch.no_grad():
    images = model.generator(z)

images = (images + 1) / 2
grid = vutils.make_grid(images, nrow=8)

plt.figure(figsize=(8,8))
plt.imshow(grid.permute(1,2,0).cpu())
plt.axis("off")
plt.show()


In [ ]:
vutils.save_image(images, "final_generated_samples.png", nrow=8)
print("Saved final_generated_samples.png")
